# XiaoNetV5B Architecture Explorer

This notebook visualizes the architecture, parameter count, and data flow of the `XiaoNetV5B` model. You can change the `base_channels` in the configuration cell below to see how it affects the model's size and complexity.

In [1]:
import torch
from models.xn_xiao_net_v5b_sigmoid import XiaoNetV5B

In [6]:
# --- Configuration ---
# Adjust the base_channels to see how the model size and layer parameters change.
# The default for Orin Nano optimization is 7.
# A value of 16 is used in some training scripts.
base_channels = 8
# ---------------------

# Instantiate model with specified parameters
#model = XiaoNetV5B(window_len=3001, in_channels=3, num_phases=3, base_channels=base_channels)
model = XiaoNetV5B(window_len=3001, in_channels=3, num_phases=3, base_channels=base_channels)
print(f"✓ Model instantiated with base_channels={base_channels}")

✓ Model instantiated with base_channels=8


## Model Summary

In [7]:
# Get model info
info = model.get_model_info()
total_params, trainable_params = model.count_parameters()

print('Model Configuration:')
print(f'  • Input: (batch, 3 channels, 3001 samples)')
print(f'  • Output: (batch, 3 phases, 3001 samples) [P, S, Noise]')
print(f'  • Base channels: {model.base_channels}')
print(f'  • Activation: Sigmoid (independent probabilities)')
print()
print('Model Statistics:')
print(f'  • Total parameters: {total_params:,}')
print(f'  • Trainable parameters: {trainable_params:,}')
print()
print('Deployment Readiness:')
print(f'  • GPU optimized: {info["gpu_optimized"]}')
print(f'  • TensorRT compatible: {info["tensorrt_compatible"]}')

Model Configuration:
  • Input: (batch, 3 channels, 3001 samples)
  • Output: (batch, 3 phases, 3001 samples) [P, S, Noise]
  • Base channels: 8
  • Activation: Sigmoid (independent probabilities)

Model Statistics:
  • Total parameters: 20,208
  • Trainable parameters: 20,208

Deployment Readiness:
  • GPU optimized: True
  • TensorRT compatible: True


## Layer-by-Layer Breakdown

In [8]:
# Helper function to count parameters in a module
def count_layer_params(module):
    return sum(p.numel() for p in module.parameters())

# Use f-strings to make the channel numbers dynamic
bc = model.base_channels

print('ENCODER (Downsampling Path):')
print(f'  enc1 (3→{bc})       : {count_layer_params(model.enc1):>6,} params | kernel=7, stride=1 | Output: ({bc}, 3001)')
print(f'  enc2 ({bc}→{bc*2})      : {count_layer_params(model.enc2):>6,} params | kernel=5, stride=4 | Output: ({bc*2}, 750)')
print(f'  enc3 ({bc*2}→{bc*4})     : {count_layer_params(model.enc3):>6,} params | kernel=5, stride=4 | Output: ({bc*4}, 187)')
print(f'  bottleneck({bc*4}→{bc*8}): {count_layer_params(model.bottleneck):>6,} params | kernel=3, stride=4 | Output: ({bc*8}, 46)')
print()

print('MULTI-SCALE FEATURE EXTRACTOR (at bottleneck):')
print(f'  multi_scale      : {count_layer_params(model.multi_scale):>6,} params | kernels=[3,5,7], parallel')
print()

print('DECODER (Upsampling Path + Skip Connections):')
print(f'  dec3 ({bc*8}→{bc*4})     : {count_layer_params(model.dec3):>6,} params | upsample×4, kernel=5 | + skip from enc3')
print(f'  dec2 ({bc*4}→{bc*2})     : {count_layer_params(model.dec2):>6,} params | upsample×4, kernel=5 | + skip from enc2')
print(f'  dec1 ({bc*2}→{bc})      : {count_layer_params(model.dec1):>6,} params | upsample×4, kernel=5 | + skip from enc1')
print()

print('OUTPUT:')
print(f'  output ({bc}→3)     : {count_layer_params(model.output):>6,} params | 1×1 conv')
print(f'  sigmoid          : activation (per-channel independence)')

ENCODER (Downsampling Path):
  enc1 (3→8)       :     61 params | kernel=7, stride=1 | Output: (8, 3001)
  enc2 (8→16)      :    200 params | kernel=5, stride=4 | Output: (16, 750)
  enc3 (16→32)     :    656 params | kernel=5, stride=4 | Output: (32, 187)
  bottleneck(32→64):  2,272 params | kernel=3, stride=4 | Output: (64, 46)

MULTI-SCALE FEATURE EXTRACTOR (at bottleneck):
  multi_scale      : 13,632 params | kernels=[3,5,7], parallel

DECODER (Upsampling Path + Skip Connections):
  dec3 (64→32)     :  2,432 params | upsample×4, kernel=5 | + skip from enc3
  dec2 (32→16)     :    704 params | upsample×4, kernel=5 | + skip from enc2
  dec1 (16→8)      :    224 params | upsample×4, kernel=5 | + skip from enc1

OUTPUT:
  output (8→3)     :     27 params | 1×1 conv
  sigmoid          : activation (per-channel independence)


## Data Flow Diagram

In [9]:
# Using f-strings to make the channel numbers dynamic
bc = model.base_channels
print('Input (3, 3001)')
print('     ↓')
print(f'enc1 → ({bc}, 3001)  ──────────────────────────┐')
print('     ↓                                       │')
print(f'enc2 → ({bc*2}, 750)  ─────────────────┐        │')
print('     ↓                              │        │')
print(f'enc3 → ({bc*4}, 187)  ───────┐         │        │')
print('     ↓                    │         │        │')
print(f'bottleneck → ({bc*8}, 46)    │         │        │')
print('     ↓                    │         │        │')
print(f'multi_scale → ({bc*8}, 46)   │         │        │')
print('     ↓                    │         │        │')
print(f'dec3 → ({bc*4}, 187) ←───────┘         │        │')
print('     ↓                              │        │')
print(f'dec2 → ({bc*2}, 750) ←─────────────────┘        │')
print('     ↓                                       │')
print(f'dec1 → ({bc}, 3001) ←──────────────────────────┘')
print('     ↓')
print('output + sigmoid → (3, 3001)')

Input (3, 3001)
     ↓
enc1 → (8, 3001)  ──────────────────────────┐
     ↓                                       │
enc2 → (16, 750)  ─────────────────┐        │
     ↓                              │        │
enc3 → (32, 187)  ───────┐         │        │
     ↓                    │         │        │
bottleneck → (64, 46)    │         │        │
     ↓                    │         │        │
multi_scale → (64, 46)   │         │        │
     ↓                    │         │        │
dec3 → (32, 187) ←───────┘         │        │
     ↓                              │        │
dec2 → (16, 750) ←─────────────────┘        │
     ↓                                       │
dec1 → (8, 3001) ←──────────────────────────┘
     ↓
output + sigmoid → (3, 3001)


## Key Features

*   **Grouped Depthwise Separable Convolutions**: Parameter efficient and GPU-friendly.
*   **Multi-scale feature extraction**: At the bottleneck, parallel paths with kernels of size 3, 5, and 7 capture features at different scales.
*   **Skip connections**: U-Net style connections preserve high-resolution information for precise phase localization.
*   **Sigmoid activation**: Allows for independent probabilities for P, S, and Noise, making it suitable for multi-label classification.
*   **GPU-optimized**: Specifically designed with NVIDIA Orin Nano deployment in mind.
*   **TensorRT ready**: The architecture uses operations compatible with TensorRT for significant inference speedup on NVIDIA hardware.